In [1]:
import pandas as pd
import os
import sys
import numpy as np
import time
import re

from scipy.stats import wilcoxon, ttest_1samp, kruskal
from statsmodels.stats.descriptivestats import sign_test

idx = pd.IndexSlice

utilsPath = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\utils'
if utilsPath not in sys.path:
    sys.path.append(utilsPath)

In [2]:
#
# Set constants
#

Qworking_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\PESA"
Mworking_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\PESA"

# X
xq = pd.read_csv(os.path.join(Qworking_path, "WorkingFiles", "Xq_minus_X_norm.tsv"), sep='\t', index_col='Seqn')
xm = pd.read_csv(os.path.join(Mworking_path, "WorkingFiles", "Xm_norm.tsv"), sep='\t', index_col='Seqn')

# Feature info
q2i = pd.read_csv(os.path.join(Qworking_path, "WorkingFiles", "q2info.tsv"), sep='\t', index_col='fid')
m2i = pd.read_csv(os.path.join(Mworking_path, "WorkingFiles", "f2i.tsv"), sep='\t', index_col='fid')

# metadata
mdata = pd.read_csv(
    os.path.join(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metadata\PESA\WorkingFiles\main_metadata.tsv"), 
    sep='\t', index_col='Seqn')

In [3]:
mdata.columns

Index(['Name', 'Cohort', 'Group', 'Global ORDER', 'Cohort ORDER',
       'Cohort Batch', 'Global_Batch', 'Plaque_thickness', 'Calcium_Score',
       'territorios afectados', 'Plaque_burden', 'Total_Cholesterol', 'HDL',
       'LDL', 'Ox-LDL', 'Lipoprotein(a)', 'CRP', 'Framingham 10y',
       'Framingham 30y', 'PESA_Score', 'Indicacion medicamento',
       'Tratamiento crÃ³nico', 'rxname', 'rxindicfrcv', 'rxindic', 'deqage',
       'deqsex', 'Systolic Blodd Pressure', 'Diastolic Blood Pressure',
       'Diabetes history', 'Smoke', 'Caso/control', 'Smoke_dummy'],
      dtype='object')

In [4]:
# Retener observaciones presentes en ambas omicas

cidx = np.intersect1d(xq.index, xm.index).tolist()
cidx = np.intersect1d(cidx, mdata.index[mdata['Total_Cholesterol']>0]).tolist()

mdatai = mdata.loc[cidx].copy()
Y = mdatai['Total_Cholesterol'].to_numpy()
Ybin = Y!=0

xqi = xq.loc[cidx].copy()
xmi = xm.loc[cidx].copy()

In [5]:
# Eliminar de metabolomica observaciones UNK

xmi = xmi[m2i[m2i.TP_ID!='UNK'].index]

In [6]:
# Numero de features y observaciones

print(f'Number of observations: {mdatai.shape[0]}')
print(f'Proteomics Features: {xqi.shape[1]}')
print(f'Metabolomic Features: {xmi.shape[1]}')

Number of observations: 364
Proteomics Features: 470
Metabolomic Features: 1186


In [7]:
# Pre-Seleccion de features para igualar ambas omicas y acelerar entrenamiento de modelos
topN = 100


from sklearn.feature_selection import f_classif

q2F = f_classif(xqi, Y)
q2F = pd.DataFrame({'F': q2F[0], 'pv': q2F[1]}, index=xqi.columns)
qfilt = q2F.sort_values('F', ascending=False).iloc[:topN, :].index.tolist()
print('Last Feature Proteomics:')
print(f'{q2F.loc[qfilt[-1], :]}')

print()

m2F = f_classif(xmi, Y)
m2F = pd.DataFrame({'F': m2F[0], 'pv': m2F[1]}, index=xmi.columns)
mfilt = m2F.sort_values('F', ascending=False).iloc[:topN, :].index.tolist()
print('Last Feature Metabolomics:')
print(f'{m2F.loc[mfilt[-1], :]}')

Last Feature Proteomics:
F     1.307860
pv    0.129485
Name: P08571, dtype: float64

Last Feature Metabolomics:
F     1.350053
pv    0.103189
Name: C18P13, dtype: float64


In [8]:
xqi = xqi.loc[:, qfilt]
xmi = xmi.loc[:, mfilt]
xqmi = xqi.join(xmi)
xfi = mdatai[['Framingham 10y']]

Test Different Machine Learning Models

In [9]:
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVC, SVR

from sklearn.model_selection import KFold, GridSearchCV, cross_val_score, StratifiedKFold, train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score,roc_auc_score

In [10]:
# Machine Learning with double CV

def CLF1(X, Y, Ybin, classifier, p_grid, search='grid'):
    models = { }

    # Data Set
    # X = xqi


    # Train / Test split
    # kfold_outer = list(StratifiedKFold(int(min(1/0.1, X.shape[0]))).split(X, Ybin))
    kfold_outer = list(KFold(int(min(1/0.1, X.shape[0]))).split(X, Y))

    #n_outer = 0
    for n_outer in range(len(kfold_outer)):

        train, test = kfold_outer[n_outer]

        X_train, Y_train, Ybin_train = X.iloc[train], Y[train], Ybin[train] 

        # k_fold_inner = StratifiedKFold(int(min(1/0.1, X_train.shape[0]))).split(X_train, Ybin_train)
        k_fold_inner = KFold(int(min(1/0.1, X_train.shape[0]))).split(X_train, Y_train)

        if search == 'grid':
            clf = GridSearchCV(
                estimator=classifier,
                param_grid=p_grid,
                cv=k_fold_inner,
                n_jobs=-1,
                verbose=2
            )
        
        elif search == 'random':
            clf = RandomizedSearchCV(
                estimator = classifier,
                param_distributions = p_grid,
                n_iter = 10, 
                cv = k_fold_inner, 
                verbose=2, 
                random_state=0, 
                n_jobs = -1
            )
        
        elif search == None:
            clf = classifier

        starttime = time.time()
        clf.fit(X_train, Y_train)

        best_estimator_ = clf.best_estimator_ if search!=None else clf

        models[n_outer] = {
            'clf': clf,
            'train': train,
            'test': test,
            # 'train_score': accuracy_score(Y[train], best_estimator_.predict(X.iloc[train])),
            # 'validation_score': clf.best_score_ if search != None else None,
            # 'test_score': accuracy_score(Y[test], best_estimator_.predict(X.iloc[test])),
            # 'test_auc': roc_auc_score(Y[test], best_estimator_.predict_proba(X.iloc[test])[:,1]),
            'time': time.time() - starttime
        }

    return models

In [11]:
#
# Classifiers
#

# Logistic Regression

LogReg = ElasticNet(
    #random_state=0,
    max_iter=1000,
)

# Hyperparameter Search Field

# C is inverse of regularization strength
Csearch = (-2,1,0.25) #(min, max, step) # Log10 Scale
Csearch = 10**np.arange(Csearch[0], Csearch[1]+Csearch[2], Csearch[2])
print('C values')
print(', '.join([str(round(i, 3)) for i in Csearch]))
print()

# L = L1/(L1+L2) ratio --> L=0 ~ L2 | L=1 ~ L1
Lsearch = (0.1, 1, 0.1)
Lsearch = np.arange(Lsearch[0], Lsearch[1]+Lsearch[2], Lsearch[2])
print('L values')
print(', '.join([str(round(i, 3)) for i in Lsearch]))
print()

p_grid_LogReg = {
    'alpha':Csearch, 'l1_ratio':Lsearch
}

# Support Vector Classifier

svc = SVR()

p_grid_svc = { 
    # Regularization parameter.
    "C": Csearch,
    # Kernel type
    "kernel": ['rbf', 'poly'],
    # Gamma is the Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
    "gamma": np.logspace(-1, 1, 3).tolist()+['scale', 'auto']
    }


# Random Forest

RFC = RandomForestRegressor()

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid_rfc = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


# Gradient Boosting

GBC = GradientBoostingRegressor()

# Learning rate
learning_rates = 10**np.arange(-2,0.25, 0.25)
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['log2', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Create the random grid
random_grid_gbc = {
    'learning_rate': learning_rates,
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
            }


C values
0.01, 0.018, 0.032, 0.056, 0.1, 0.178, 0.316, 0.562, 1.0, 1.778, 3.162, 5.623, 10.0

L values
0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0



In [12]:
Models = {'q': {}, 'm': {}, 'qm': {}, 's': {}}

# Proteomics
qmodels = {}
qmodels['lr'] = CLF1(xqi, Y, Ybin, LogReg, p_grid_LogReg, 'grid')
qmodels['sv'] = CLF1(xqi, Y, Ybin, svc, p_grid_svc, 'grid')
qmodels['rf'] = CLF1(xqi, Y, Ybin, RFC, random_grid_rfc, 'random')
qmodels['gb'] = CLF1(xqi, Y, Ybin, GBC, random_grid_gbc, 'random')

Models['q'] = qmodels


# Metabolomics
mmodels = {}
mmodels['lr'] = CLF1(xmi, Y, Ybin, LogReg, p_grid_LogReg, 'grid')
mmodels['sv'] = CLF1(xmi, Y, Ybin, svc, p_grid_svc, 'grid')
mmodels['rf'] = CLF1(xmi, Y, Ybin, RFC, random_grid_rfc, 'random')
mmodels['gb'] = CLF1(xmi, Y, Ybin, GBC, random_grid_gbc, 'random')

Models['m'] = mmodels


# Proteomics & Metabolomics

qmmodels = {}
qmmodels['lr'] = CLF1(xqmi, Y, Ybin, LogReg, p_grid_LogReg, 'grid')
qmmodels['sv'] = CLF1(xqmi, Y, Ybin, svc, p_grid_svc, 'grid')
qmmodels['rf'] = CLF1(xqmi, Y, Ybin, RFC, random_grid_rfc, 'random')
qmmodels['gb'] = CLF1(xqmi, Y, Ybin, GBC, random_grid_gbc, 'random')

Models['qm'] = qmmodels

Fitting 10 folds for each of 130 candidates, totalling 1300 fits


Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each

KeyboardInterrupt: 

In [ ]:
# Stacked Model
LogReg = ElasticNet(
    max_iter=100000,
)

# Hyperparameter Search Field

# C is inverse of regularization strength
Csearch = (-2,1,0.25) #(min, max, step) # Log10 Scale
Csearch = 10**np.arange(Csearch[0], Csearch[1]+Csearch[2], Csearch[2])

# L = L1/(L1+L2) ratio --> L=0 ~ L2 | L=1 ~ L1
Lsearch = (0.1, 1, 0.1)
Lsearch = np.arange(Lsearch[0], Lsearch[1]+Lsearch[2], Lsearch[2])


p_grid_LogReg = {
    'alpha':Csearch, 'l1_ratio':Lsearch
}

In [ ]:
model = 'lr'

def ENStacked(modelq, modelm, xqi, xmi, x=None):

    stacked = {}

    for i in modelq:

        submodelq = modelq[i]
        submodelm = modelm[i]
        train, test = submodelq['train'], submodelq['test']

        qlogit = submodelq['clf'].best_estimator_.predict(xqi)
        mlogit = submodelm['clf'].best_estimator_.predict(xmi)

        # Logit > 0 --> Disease
        logit = pd.DataFrame({
            'qlogit': qlogit, #qlogit[:, 1]-qlogit[:, 0],
            'mlogit': mlogit #mlogit[:, 1]-mlogit[:, 0]
        })

        if type(x)!=type(None):
            logit = logit.join(x.reset_index(drop=True))

        # k_fold_inner = StratifiedKFold(int(min(1/0.1, train.shape[0]))).split(logit.iloc[train], Ybin[train])
        k_fold_inner = KFold(int(min(1/0.1, train.shape[0]))).split(logit.iloc[train], Y[train])

        clf = GridSearchCV(
            estimator=LogReg,
            param_grid=p_grid_LogReg,
            cv=k_fold_inner,
            n_jobs=-1,
            verbose=2
        )

        starttime = time.time()
        clf.fit(logit.iloc[train], Y[train])

        stacked[i] = {
            'clf': clf,
            'train': train,
            'test': test,
            'logit': logit,
            # 'train_score': accuracy_score(Y[train], clf.best_estimator_.predict(logit.iloc[train])),
            # 'validation_score': clf.best_score_ ,
            # 'test_score': accuracy_score(Y[test], clf.best_estimator_.predict(logit.iloc[test])),
            'time': time.time() - starttime
        }
    
    return stacked

In [ ]:
smodels = {}
for i in ['lr', 'sv', 'rf', 'gb']:
    print(i)
    smodels[i] = ENStacked(Models['q'][i], Models['m'][i], xqi, xmi)
    print()

Models['s'] = smodels

lr
Fitting 10 folds for each of 130 candidates, totalling 1300 fits


Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits



In [ ]:
fModels = {}
for i in ['lr', 'sv', 'rf', 'gb']:
    print(i)
    fModels[i] = ENStacked(Models['q'][i], Models['m'][i], xqi, xmi, xfi)
    print()

lr
Fitting 10 folds for each of 130 candidates, totalling 1300 fits


Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits
Fitting 10 folds for each of 130 candidates, totalling 1300 fits



In [ ]:
# for i in ['lr', 'sv', 'rf', 'gb']:
#     print(i)
#     fmodels['s'][i] = ENStacked(Models['q'][i], Models['m'][i], xqi, xmi, xfi)
#     print()

In [ ]:
# import pickle

# with open('Models_continuous.pkl', 'wb') as file:
#     pickle.dump(Models, file)

# with open('fmodels_continuous.pkl', 'wb') as file:
#     pickle.dump(fmodels, file)

In [ ]:
# import pickle

# with open('Models_continuous.pkl', 'rb') as file:
#     Models = pickle.load(file)

# with open('fmodels.pkl', 'rb') as file:
#     fmodels = pickle.load(file)

Framingham predictions

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
LogReg = LinearRegression(
    n_jobs=-1,
)

fhsonly = CLF1(xfi, Y, Ybin, LogReg, {}, None)

In [ ]:
from scipy.stats import pearsonr, spearmanr

In [ ]:
fhsonly_score = {'r2':[], 'corr_r':[], 'corr_pv':[], 'log_corr_pv':[]}

for key, value in fhsonly.items():

    logit = value['clf'].predict(xfi.iloc[value['test']])

    k = spearmanr(Y[value['test']],logit)
    fhsonly_score['corr_r'].append(k.statistic)
    fhsonly_score['corr_pv'].append(k.pvalue)
    fhsonly_score['log_corr_pv'].append(-np.log10(k.pvalue))

    fhsonly_score['r2'].append(
        value['clf'].score(xfi.iloc[value['test']],Y[value['test']])
    )

SignificanceResult(statistic=0.007112375533428163, pvalue=0.9666752989327565)
34955.28243922836
SignificanceResult(statistic=0.2835467045993362, pvalue=0.08900834718226774)
22266.215024705994
SignificanceResult(statistic=0.23660502607871028, pvalue=0.15856155177635192)
12527.333687577553
SignificanceResult(statistic=0.3183165404705149, pvalue=0.0548498356878494)
5975.833505118368
SignificanceResult(statistic=0.34442370880253276, pvalue=0.039683813548166164)
1571.2380779096693
SignificanceResult(statistic=0.2775131957601095, pvalue=0.10127932747949948)
189.9662514376046
SignificanceResult(statistic=0.15546975546975547, pvalue=0.36524275937849215)
1157.7955811873903
SignificanceResult(statistic=0.5044082641244865, pvalue=0.0017072158128993771)
5192.806547771563
SignificanceResult(statistic=0.23259106898818085, pvalue=0.17222092735507252)
10807.288780779923
SignificanceResult(statistic=0.5067576306621294, pvalue=0.001610777987393265)
19646.203318922766


Scores

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
palette = px.colors.qualitative.Plotly


In [ ]:
from sklearn.metrics import r2_score, explained_variance_score

In [ ]:
def getScores(Models, Xdata):
    modelScores = {}
    for name in Models:
        modelScores.update({name:{}})

        myclf = 'lr'
        for myclf in Models[name]:
            modelScores[name][myclf] = {'r2':[], 'exVar':[], 'time':[], 'corr_r':[], 'corr_pv':[], 'log_corr_pv':[]}
            for i in range(len(Models[name][myclf].keys())):

                train, test = Models[name][myclf][i]['train'], Models[name][myclf][i]['test']

                if name == 's':
                    mylogit = Models[name][myclf][i]['clf'].best_estimator_.predict(Models[name][myclf][i]['logit'].iloc[test])
                else:
                    mylogit = Models[name][myclf][i]['clf'].best_estimator_.predict(Xdata[name].iloc[test])
                
                k = spearmanr(Y[test], mylogit)
                modelScores[name][myclf]['corr_r'].append(k.statistic)
                modelScores[name][myclf]['corr_pv'].append(k.pvalue)
                modelScores[name][myclf]['log_corr_pv'].append(-np.log10(k.pvalue))

                modelScores[name][myclf]['r2'].append(
                    r2_score(Y[test], mylogit)
                )
                modelScores[name][myclf]['exVar'].append(
                    explained_variance_score(Y[test], mylogit)
                )

                modelScores[name][myclf]['time'].append(Models[name][myclf][i]['time'])
    return modelScores

In [ ]:
Xdata = {
    'q': xqi, 'm': xmi, 'qm': xqmi
}
modelScores = getScores(Models, Xdata)

In [ ]:
fmodelScores = getScores({'s': fModels}, Xdata)

Plots

In [41]:
fig = make_subplots(rows=1, cols=3, subplot_titles=['R2', '-LogPval'])

i = 'q'

mymodels = ['lr']#['rf', 'gb', 'sv', 'lr']

fig.add_trace(go.Bar(
    x=['FHS10y'], y=[np.median(fhsonly_score['r2'])], marker_color='black',offset=-0.15, width=0.2, opacity=0.8,
    error_y=dict(
        array=[np.median(np.abs(np.array(fhsonly_score['r2'])-np.median(fhsonly_score['r2']))) for j in mymodels]
    )
), row=1, col=1)

fig.add_trace(go.Bar(
    x=['FHS10y'], y=[np.median(-np.log(fhsonly_score['pv']))], marker_color='black',offset=-0.15, width=0.2, opacity=0.8,
    error_y=dict(
        array=[np.median(np.abs(np.array(-np.log(fhsonly_score['pv']))-np.median(-np.log(fhsonly_score['pv'])))) for j in mymodels]
    )
), row=1, col=2)


for n,i in enumerate(['q', 'm', 'qm', 's']):
    fig.add_trace(go.Bar(
        x=[i.upper() for i in mymodels],
        y=[np.median(modelScores[i][j]['r2']) for j in mymodels],
        opacity=0.8, name=i, marker_color=palette[n], legendgroup=n,#, offsetgroup=n+1
        error_y=dict(
            array=[np.median(np.abs(np.array(modelScores[i][j]['r2'])-np.median(modelScores[i][j]['r2']))) for j in mymodels]
        )
    ), row=1, col=1)

    fig.add_trace(go.Bar(
        x=[i.upper() for i in mymodels],
        y=[np.mean(-np.log(modelScores[i][j]['pv'])) for j in mymodels],
        opacity=0.8, name=i, marker_color=palette[n], legendgroup=n,#, offsetgroup=n+1
        error_y=dict(
            array=[np.std(-np.log(modelScores[i][j]['pv'])) for j in mymodels]
            #array=[np.median(np.abs(np.array(-np.log(modelScores[i][j]['pv']))-np.median(-np.log(modelScores[i][j]['pv'])))) for j in mymodels]
        )
    ), row=1, col=2)

#     fig.add_trace(go.Bar(
#         x=[i.upper() for i in mymodels],
#         y=[np.median(modelScores[i][j]['auc'])/np.median(np.abs(np.array(modelScores[i][j]['auc'])-np.median(modelScores[i][j]['auc']))) for j in mymodels],
#         opacity=0.8, name=i, marker_color=palette[n], legendgroup=n,#, offsetgroup=n+1
#     ), row=1, col=2)


# for n,i in enumerate(['q', 'm', 'qm', 's']):
#     fig.add_trace(go.Bar(
#         x=[i.upper() for i in mymodels],
#         y=[np.median(-np.log10(modelScores[i][j]['pv'])) for j in mymodels],
#         opacity=0.8, name=i, marker_color=palette[n], legendgroup=n,
#         error_y=dict(
#             array=[np.median(np.abs(np.array(-np.log10(modelScores[i][j]['pv']))-np.median(-np.log10(modelScores[i][j]['pv'])))) for j in mymodels]
#         )
#     ), row=1, col=4)

# for n,i in enumerate(['q', 'm', 'qm', 's']):
#     fig.add_trace(go.Bar(
#         x=[i.upper() for i in mymodels],
#         #y=[scores.loc[:, idx[:, [j]]].droplevel(axis=1, level=1).loc['test'][i][0] for j in mymodels],
#         y=[np.median(modelScores[i][j]['accuracy']) for j in mymodels],
#         opacity=0.8, name=i, marker_color=palette[n], legendgroup=n,
#         error_y=dict(
#             array=[np.median(np.abs(np.array(modelScores[i][j]['accuracy'])-np.median(modelScores[i][j]['accuracy']))) for j in mymodels]
#         )
#     ), row=1, col=3)

# fig.update_yaxes(range=(0, 1), row=1, col=1)
# fig.update_yaxes(range=(0, 1), row=1, col=3)

fig.show()

In [22]:
from scikit_posthocs import posthoc_dunn,posthoc_tukey
from statsmodels.stats.multitest import multipletests

In [21]:
res = []
for i in ['rf', 'gb', 'sv', 'lr']:
    for j in ['q', 'm', 'qm']:
        res.append([i,j,wilcoxon(np.array(modelScores[j][i]['auc'])-np.array(fhsonly_score['auc']), method='approx', zero_method='zsplit')])
        # res.append([i,j,ttest_1samp(np.array(modelScores[j][i]['auc'])-np.array(fhsonly_score['auc']),0)])
        print(res[-1])
    print()
        

['rf', 'q', WilcoxonResult(statistic=20.0, pvalue=0.4445867389117455)]
['rf', 'm', WilcoxonResult(statistic=8.0, pvalue=0.04685328478814715)]
['rf', 'qm', WilcoxonResult(statistic=6.0, pvalue=0.028314054945765665)]

['gb', 'q', WilcoxonResult(statistic=25.5, pvalue=0.8384637819224636)]
['gb', 'm', WilcoxonResult(statistic=16.0, pvalue=0.24112136827741826)]
['gb', 'qm', WilcoxonResult(statistic=9.0, pvalue=0.05933611988090862)]

['sv', 'q', WilcoxonResult(statistic=20.0, pvalue=0.4445867389117455)]
['sv', 'm', WilcoxonResult(statistic=7.0, pvalue=0.03665792867221451)]
['sv', 'qm', WilcoxonResult(statistic=8.0, pvalue=0.04685328478814715)]

['lr', 'q', WilcoxonResult(statistic=19.5, pvalue=0.4148230671847747)]
['lr', 'm', WilcoxonResult(statistic=7.0, pvalue=0.03665792867221451)]
['lr', 'qm', WilcoxonResult(statistic=3.0, pvalue=0.012515318690073973)]



In [23]:
[(i[0], i[1], j) for i,j in zip(res,multipletests([i[2].pvalue for i in res], method='fdr_bh')[1])]

[('rf', 'q', 0.48500371517644963),
 ('rf', 'm', 0.0937065695762943),
 ('rf', 'qm', 0.0937065695762943),
 ('gb', 'q', 0.8384637819224636),
 ('gb', 'm', 0.3616820524161274),
 ('gb', 'qm', 0.10171906265298619),
 ('sv', 'q', 0.48500371517644963),
 ('sv', 'm', 0.0937065695762943),
 ('sv', 'qm', 0.0937065695762943),
 ('lr', 'q', 0.48500371517644963),
 ('lr', 'm', 0.0937065695762943),
 ('lr', 'qm', 0.0937065695762943)]

In [25]:
i = 'lr'
for i in ['rf', 'gb', 'sv', 'lr']:
    print(i.upper())
    print(kruskal(
        modelScores['q'][i]['auc'],
        modelScores['m'][i]['auc'],
        modelScores['qm'][i]['auc']    
    ))

    print(posthoc_dunn([
        modelScores['q'][i]['auc'],
        modelScores['m'][i]['auc'],
        modelScores['qm'][i]['auc']
    ],p_adjust='fdr_bh'))
    print()

RF
KruskalResult(statistic=3.0871800578677875, pvalue=0.21361284692304883)
          1         2         3
1  1.000000  0.420386  0.245425
2  0.420386  1.000000  0.508902
3  0.245425  0.508902  1.000000

GB
KruskalResult(statistic=2.6096128170894533, pvalue=0.2712250368130723)
          1         2         3
1  1.000000  0.482740  0.328543
2  0.482740  1.000000  0.542081
3  0.328543  0.542081  1.000000

SV
KruskalResult(statistic=2.4232258064515975, pvalue=0.2977167037411928)
          1         2         3
1  1.000000  0.334152  0.334152
2  0.334152  1.000000  0.819180
3  0.334152  0.819180  1.000000

LR
KruskalResult(statistic=7.161290322580641, pvalue=0.02785771973393869)
          1         2         3
1  1.000000  0.191262  0.022959
2  0.191262  1.000000  0.253038
3  0.022959  0.253038  1.000000



Análisis de los coeficientes

In [37]:
mdl = 'lr'

Mcoefs = {}

for j in ['q', 'm', 'qm']:
    Mcoefs[j] = pd.DataFrame([
        i['clf'].best_estimator_.coef_[0]
        for i in Models[j][mdl].values()
    ], columns=Xdata[j].columns)

In [43]:
list(Models['qm']['lr'].values())[0]['clf'].best_estimator_.coef_[0]
wilcoxon(Mcoefs['q'], method='approx', zero_method='zsplit').pvalue.shape

(100,)

In [45]:

MCscore = {}
for j in ['q', 'm', 'qm']:
    MCscore[j] = pd.DataFrame()
    MCscore[j]['median'] = Mcoefs[j].median()
    MCscore[j]['medianAbs'] = Mcoefs[j].median().abs()
    MCscore[j]['mad'] = (Mcoefs[j]-Mcoefs[j].median()).abs().median()
    MCscore[j]['pv'] = wilcoxon(
        Mcoefs[j],
        # pd.DataFrame([
        #     i['clf'].best_estimator_.coef_[0]
        #     for i in Models[j][mdl].values()
        # ]), 
        method='approx', zero_method='zsplit'
    ).pvalue
    MCscore[j]['qv'] = multipletests(MCscore[j]['pv'], method='fdr_bh')[1]

In [410]:
# Check coefs

i = MCscore['qm'].sort_values('pv').join(q2i).join(m2i[['TP_ID']])
i.to_excel('LR_qm_coefs.xlsx')

In [46]:
i1 = pd.DataFrame({'shortName': [
j.groups()[0] if j else i.groups()[0]
for i,j in zip(
    [re.search(r'\|([^|]+)\|', i) for i in q2i.qdesc],
    [re.search(r'GN=([^\s]+)', i) for i in q2i.qdesc]
)
]})
i1['type'] = 'q'
i1['id'] = q2i.index

i2 = pd.DataFrame({'shortName':[
    j.groups()[0] if j else i
    for i,j in zip(
        [ [j.split(' | ')[ np.argmin([len(k) for k in j.split(' | ')]) ]  for j in i.split(' // ') ][0] for i in m2i.TP_ID],
        [re.search(r'LMSD{ ([^}]+) }', i) for i in m2i.TP_ID]
    )
]})
i2['type'] = 'm'
i2['id'] = m2i.index

shortName = pd.concat([i1, i2]).set_index('id')

# [re.search(r'LMSD{ ([^}]+) }', i) for i in m2i.TP_ID]
# m2i.TP_ID.tolist()

In [47]:
# Representar boxplot con coeficientes de qq y mm significativos
qvThr = 0.05
pvThr = 0.01

i = 'qm'
j = 'lr'

for i in ['q', 'm', 'qm']:
    MCscore[i]['sig'] = np.logical_and(MCscore[i].qv < qvThr, MCscore[i].pv < pvThr)
    MCscore[i] = MCscore[i].sort_values('median')

In [48]:
omic = 'qm'

fig = go.Figure()

[
    fig.add_trace(go.Box(
        x=coefs,
        name=f"{shortName.loc[feature, 'shortName']} | {feature}",
        marker_color=palette[0] if shortName.loc[feature, 'type']=='q' else palette[1],
        legendgroup=0 if shortName.loc[feature, 'type']=='q' else 1
    ))
    for feature, coefs in Mcoefs[omic][MCscore[omic].index[MCscore[omic].sig]].items()
]

fig.update_yaxes(showgrid=True)
fig.update_layout(height=2000, width=900)
fig.show()

with open('Coef_distribution.html', 'w') as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn', default_height='50%', default_width='80%'))

In [50]:
# Calcular AUC para m y q significativas

omic = 'qm'
MCmodels = {}

for omic in ['q', 'm', 'qm']:
    LogReg = LogisticRegression(
        penalty='none',
        verbose=2,
        n_jobs=-1,
        random_state=0
    )

    # CLF1(xfi, Y, LogReg, {}, None)
    MCmodels[omic] = [
        CLF1(
            xqmi[i].to_frame(),
            Y,
            LogReg,
            {},
            None
        )
        for i in MCscore[omic].index
    ]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1

In [53]:
MCMauc = {}

omic = 'qm'
for omic in ['q', 'm', 'qm']:
    MCMauc[omic] = pd.DataFrame([[j['test_auc'] for j in i.values()] for i in MCmodels[omic]], index=MCscore[omic].index).T
    MCscore[omic]['auc_median'] = MCMauc[omic].median()
    MCscore[omic]['auc_mad'] = np.abs(MCMauc[omic]-MCMauc[omic].median()).median()


In [54]:
MCscore['qm']

,median,medianAbs,mad,pv,qv,sig,auc_median,auc_mad
HILP113,-0.206555,0.206555,0.031228,0.005062,0.013771,True,0.593080,0.024935
HILP60,-0.198957,0.198957,0.061527,0.005062,0.013771,True,0.572531,0.055556
C18P612,-0.197183,0.197183,0.065429,0.005062,0.013771,True,0.641813,0.054906
HILP202,-0.181527,0.181527,0.039529,0.005062,0.013771,True,0.585608,0.059292
Q9Y6R7,-0.180209,0.180209,0.062651,0.005062,0.013771,True,0.585770,0.024204
...,...,...,...,...,...,...,...,...
C18P654,0.148519,0.148519,0.032228,0.005062,0.013771,True,0.586420,0.032164
Q9UL89,0.167654,0.167654,0.077823,0.005062,0.013771,True,0.577485,0.054012
P04114,0.182543,0.182543,0.052741,0.005062,0.013771,True,0.596248,0.055556
C18P463,0.183087,0.183087,0.097944,0.005062,0.013771,True,0.545809,0.108025


In [436]:
shortName

,shortName,type
id,,
P04114,APOB,q
V9HWA9,HEL-S-62p,q
P02768,ALB,q
P0C0L4,C4A,q
P01023,A2M,q
...,...,...
HILP583,UNK,m
HILP584,UNK,m
HILP585,UNK,m


In [59]:
# Compare omic auc with single feature

omic = 'qm'
n = 50

fig = go.Figure()

aucref = np.array(modelScores[omic]['lr']['auc'])
fig.add_trace(go.Bar(
    x=[omic],
    y=[np.median(aucref)],
    error_y=dict(type='data',array=[np.median(np.abs(np.median(aucref)-aucref))]),
    marker_color='black', opacity=0.7
))

tmp = MCscore[omic].sort_values(['pv','medianAbs'], ascending=[True,False])

fig.add_trace(go.Bar(
    x= [f'{i} | {j}' for i,j in zip(shortName.loc[tmp.index[:n]].shortName, shortName.index.tolist())],
    y=tmp['auc_median'][:n],
    marker_color = [palette[0] if shortName.loc[i, 'type']=='q' else palette[1] for i in tmp.index],
    error_y=dict(array=tmp['auc_mad'][:n])
))

fig.update_yaxes(range=(0.4,1))
fig.update_layout(height=700)

AUC del modelo completo eliminando features

In [87]:
# Logistic Regression

LogReg = LogisticRegression(
    penalty='elasticnet',
    random_state=0,
    max_iter=100,
    solver='saga'
)

# Hyperparameter Search Field

# C is inverse of regularization strength
Csearch = (-2,1,0.25) #(min, max, step) # Log10 Scale
Csearch = 10**np.arange(Csearch[0], Csearch[1]+Csearch[2], Csearch[2])

# L = L1/(L1+L2) ratio --> L=0 ~ L2 | L=1 ~ L1
Lsearch = (0, 1, 0.1)
Lsearch = np.arange(Lsearch[0], Lsearch[1]+Lsearch[2], Lsearch[2])

p_grid_LogReg = {
    'C':Csearch, 'l1_ratio':Lsearch
}

p_grid_LogReg


{'C': array([ 0.01      ,  0.01778279,  0.03162278,  0.05623413,  0.1       ,
         0.17782794,  0.31622777,  0.56234133,  1.        ,  1.77827941,
         3.16227766,  5.62341325, 10.        ]),
 'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])}

In [93]:
N = 50

mdl = 'lr'
omic = 'qm'

fset = {}
for omic in ['q', 'm', 'qm']:
    f = MCscore[omic].sort_values(['pv', 'medianAbs'], ascending=[True, False]).index.tolist()
    fset[omic] = [f[i:] for i in range(len(f)) if i<N]

In [95]:
rmModels = {
    omic: [
        CLF1(xqmi[i], Y, LogReg, p_grid_LogReg, 'grid')
        for i in fset[omic]
    ]
    for omic in ['q', 'm', 'qm']
}

Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits


c:\Users\rbarreror\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits


c:\Users\rbarreror\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits


...

Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each

c:\Users\rbarreror\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each

c:\Users\rbarreror\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits
Fitting 10 folds for each of 143 candidates, totalling 1430 fits


In [96]:
with open('rmModels.pkl', 'wb') as file:
    pickle.dump(rmModels, file)

In [ ]:
with open('rmModels.pkl', 'rb') as file:
    rmModels = pickle.load(file)

In [107]:
omic = 'qm'

rmAUC = {
    omic: [
        [j['test_auc'] for j in i.values()] for i in rmModels[omic]
    ]
    for omic in ['q', 'm', 'qm']
}

In [179]:
omic = 'm'

fig =  make_subplots(rows=1, cols=2, subplot_titles=['AUC transition', 'AUC difference'])

for n,omic in enumerate(['q', 'm', 'qm']):
    fig.add_trace(go.Scatter(
        x=np.arange(0, len(rmAUC[omic])),
        y = [np.mean(i) for i in rmAUC[omic]],
        mode='markers+lines', marker_size=4, line_width=1,
        name=omic, marker_color=palette[n],
        # error_y=dict(array=[np.std(i) for i in rmAUC[omic]])
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=np.arange(0, len(rmAUC[omic])),
        y = [np.mean(i)-np.mean(rmAUC[omic][0]) for i in rmAUC[omic]],
        mode='markers+lines', marker_size=4, line_width=1,
        name=omic, marker_color=palette[n]
    ), row=1, col=2)

fig.update_xaxes(title='Iteration')
fig.update_yaxes(range=(0.5,0.8), row=1, col=1)
fig.update_yaxes(range=(-0.2,0.01), row=1, col=2)
fig.update_layout(width=1200, height=600)

In [32]:
from functools import reduce

# Estudiar coeficientes de los modelos de LR en q, m y qm

mdl = 'lr'

# Models['q'][mdl][0]['clf'].best_estimator_.coef_

Mcoef = {}

Mcoef['q'] = pd.DataFrame(
    reduce(
        lambda x,y: x+y,
        [
            i['clf'].best_estimator_.coef_
            for i in Models['q'][mdl].values()
        ]
    )/len(Models['q'][mdl].values()), columns=xqi.columns, index=['coef']
).T.sort_values('coef', ascending=False)

In [33]:
i = Mcoef['q']

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=['Proteomics & Metabolomics', 'Proteomics & Metabolomics & FHS10y'])

mymodels = ['rf', 'gb', 'sv', 'lr']
coefs = {i:
    [
        k #np.mean(k) 
        for k in zip(*[j['clf'].best_estimator_.coef_.tolist()[0] for j in Models['s'][i].values()])
    ]
    for i in Models['s']
}

fig.add_trace(go.Bar(
    x=[i.upper() for i in mymodels],
    y=[np.mean(coefs[i][0]) for i in mymodels],
    name='q', marker_color=palette[0]
), row=1, col=1)
fig.add_trace(go.Bar(
    x=[i.upper() for i in mymodels],
    y=[np.mean(coefs[i][1]) for i in mymodels],
    name='m', marker_color=palette[1]
), row=1, col=1)

fcoefs = {i:
    [
        k #np.mean(k) 
        for k in zip(*[j['clf'].best_estimator_.coef_.tolist()[0] for j in fmodels['s'][i].values()])
    ]
    for i in mymodels
}

fig.add_trace(go.Bar(
    x=[i.upper() for i in mymodels],
    y=[np.mean(fcoefs[i][0]) for i in mymodels],
    name='q', marker_color=palette[0]
), row=1, col=2)
fig.add_trace(go.Bar(
    x=[i.upper() for i in mymodels],
    y=[np.mean(fcoefs[i][1]) for i in mymodels],
    name='m', marker_color=palette[1]
), row=1, col=2)
fig.add_trace(go.Bar(
    x=[i.upper() for i in mymodels],
    y=[np.mean(fcoefs[i][2]) for i in mymodels],
    name='FHS10y', marker_color=palette[2]
), row=1, col=2)

fig.update_layout(width=1000, title='Stacked Model Coefficients')
fig.show()
# coefs